In [5]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [6]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
936,This movie is all about entertainment. Imagine...,positive
1,I switched this on (from cable) on a whim and ...,positive
609,1st watched 12/24/2009  4 out of 10 (Dir-Robe...,negative
257,Possibly the worst film within the genre in ex...,negative
207,The Haunting is a film that boasts a really cr...,positive


In [7]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [10]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
df = normalize_text(df)
df.head()

[nltk_data] Downloading package stopwords to C:\Users\SAGAR
[nltk_data]     V\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\SAGAR
[nltk_data]     V\AppData\Roaming\nltk_data...


,review,sentiment
936,movie entertainment imagine friend love spendi...,positive
1,switched cable whim treated quite surprise alt...,positive
609,st watched  dir robert elli miller emotional ...,negative
257,possibly worst film within genre existence ann...,negative
207,haunting film boast really creepy house good e...,positive


In [11]:
df['sentiment'].value_counts()

sentiment
negative    265
positive    235
Name: count, dtype: int64

In [12]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [13]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
936,movie entertainment imagine friend love spendi...,1
1,switched cable whim treated quite surprise alt...,1
609,st watched  dir robert elli miller emotional ...,0
257,possibly worst film within genre existence ann...,0
207,haunting film boast really creepy house good e...,1


In [14]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [22]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [24]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/naveensddp1/naveen-capstone-project.mlflow')
dagshub.init(repo_owner='naveensddp1', repo_name='naveen-capstone-project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")



2025-12-27 15:19:19,106 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/naveensddp1/naveen-capstone-project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "naveensddp1/naveen-capstone-project"

2025-12-27 15:19:19,116 - INFO - Initialized MLflow to track repo "naveensddp1/naveen-capstone-project"


Repository naveensddp1/naveen-capstone-project initialized!

2025-12-27 15:19:19,119 - INFO - Repository naveensddp1/naveen-capstone-project initialized!


<Experiment: artifact_location='mlflow-artifacts:/524771347941444ca2cb7ede9feb289d', creation_time=1766828058171, experiment_id='0', last_update_time=1766828058171, lifecycle_stage='active', name='Logistic Regression Baseline', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [26]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-12-27 15:20:32,213 - INFO - Starting MLflow run...
2025-12-27 15:20:32,594 - INFO - Logging preprocessing parameters...
2025-12-27 15:20:33,735 - INFO - Initializing Logistic Regression model...
2025-12-27 15:20:33,735 - INFO - Fitting the model...
2025-12-27 15:20:33,780 - INFO - Model training complete.
2025-12-27 15:20:33,782 - INFO - Logging model parameters...
2025-12-27 15:20:34,175 - INFO - Making predictions...
2025-12-27 15:20:34,181 - INFO - Calculating evaluation metrics...
2025-12-27 15:20:34,196 - INFO - Logging evaluation metrics...
2025-12-27 15:20:35,940 - INFO - Saving and logging the model...
2025/12/27 15:20:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-12-27 15:20:46,606 - INFO - Model training and logging completed in 14.01 seconds.
2025-12-27 15:20:46,607 - INFO - Accuracy: 0.624
2025-12-27 15:20:46,607 - INFO - Precision: 0.5964912280701754
2025-12-27 15:20:46,607 - INFO - Recall: 0.5862068965517241
2025-12-27

🏃 View run popular-squid-717 at: https://dagshub.com/naveensddp1/naveen-capstone-project.mlflow/#/experiments/0/runs/cef911b0092f4c5797d51a98d4d94aa1
🧪 View experiment at: https://dagshub.com/naveensddp1/naveen-capstone-project.mlflow/#/experiments/0
